# 🧹 Masterclass de Limpieza de Datos con Pandas - Parte 2

## Contenido
1. [Limpieza de Fechas con Regex](#1)
2. [Estandarización de Categorías](#2)
3. [Detección y Manejo de Outliers](#3)
4. [Validación de Datos](#4)

En esta segunda parte nos centraremos en técnicas más avanzadas de limpieza de datos, especialmente en el manejo de fechas y categorías.

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import seaborn as sns

# Configuraciones
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

<a id='1'></a>
## 1. Limpieza de Fechas con Regex

Vamos a crear un dataset con fechas en diferentes formatos y aprenderemos a estandarizarlas:

In [ ]:
# Crear datos de ejemplo con fechas en diferentes formatos
fechas_ejemplo = pd.DataFrame({
    'fecha': [
        '2023-01-15',          # formato ISO
        '15/01/2023',          # formato europeo
        '01-15-23',            # formato americano abreviado
        '15-ene-2023',         # formato con mes en texto
        '2023.01.15',          # formato con puntos
        '15 enero 2023',       # formato texto español
        '2023/01/15 08:30:00', # formato con hora
        '15-Jan-2023',         # formato inglés
        'Jan 15, 2023',        # formato americano texto
        '20230115'             # formato compacto
    ]
})

print("Fechas originales:")
print(fechas_ejemplo)

In [ ]:
def limpiar_fecha(fecha_str):
    """Función para estandarizar fechas usando regex"""
    
    # Diccionario de meses en diferentes idiomas
    meses = {
        'ene': '01', 'jan': '01', 'enero': '01', 'january': '01',
        'feb': '02', 'february': '02', 'febrero': '02',
        'mar': '03', 'march': '03', 'marzo': '03',
        'abr': '04', 'apr': '04', 'abril': '04', 'april': '04',
        'may': '05', 'mayo': '05',
        'jun': '06', 'june': '06', 'junio': '06',
        'jul': '07', 'july': '07', 'julio': '07',
        'ago': '08', 'aug': '08', 'agosto': '08', 'august': '08',
        'sep': '09', 'september': '09', 'septiembre': '09',
        'oct': '10', 'october': '10', 'octubre': '10',
        'nov': '11', 'november': '11', 'noviembre': '11',
        'dic': '12', 'dec': '12', 'december': '12', 'diciembre': '12'
    }
    
    # Convertir a string si no lo es
    fecha_str = str(fecha_str).lower().strip()
    
    # Remover hora si existe
    fecha_str = re.sub(r'\s+\d{1,2}:\d{2}:\d{2}.*$', '', fecha_str)
    
    # Patrón para formato compacto YYYYMMDD
    if re.match(r'^\d{8}$', fecha_str):
        return f"{fecha_str[:4]}-{fecha_str[4:6]}-{fecha_str[6:]}"
    
    # Reemplazar varios separadores por -
    fecha_str = re.sub(r'[./\s]', '-', fecha_str)
    
    # Extraer componentes usando diferentes patrones
    patrones = [
        # YYYY-MM-DD
        r'^(\d{4})-(\d{1,2})-(\d{1,2})$',
        # DD-MM-YYYY
        r'^(\d{1,2})-(\d{1,2})-(\d{4})$',
        # DD-MMM-YYYY
        r'^(\d{1,2})-([a-z]{3,})-?(\d{4})$',
        # MM-DD-YY
        r'^(\d{1,2})-(\d{1,2})-(\d{2})$'
    ]
    
    for patron in patrones:
        match = re.match(patron, fecha_str)
        if match:
            grupos = match.groups()
            
            # Caso YYYY-MM-DD
            if len(grupos[0]) == 4:
                año = grupos[0]
                mes = grupos[1].zfill(2)
                dia = grupos[2].zfill(2)
            # Caso DD-MM-YYYY
            elif len(grupos[2]) == 4:
                año = grupos[2]
                # Verificar si el segundo grupo es un mes en texto
                if grupos[1].isalpha():
                    mes = meses.get(grupos[1][:3], '01')
                else:
                    mes = grupos[1].zfill(2)
                dia = grupos[0].zfill(2)
            # Caso MM-DD-YY
            else:
                año = f"20{grupos[2]}" if int(grupos[2]) < 50 else f"19{grupos[2]}"
                mes = grupos[0].zfill(2)
                dia = grupos[1].zfill(2)
            
            return f"{año}-{mes}-{dia}"
    
    return None

# Aplicar la función a nuestro DataFrame
fechas_ejemplo['fecha_limpia'] = fechas_ejemplo['fecha'].apply(limpiar_fecha)

print("\nFechas después de la limpieza:")
print(fechas_ejemplo)

<a id='2'></a>
## 2. Estandarización de Categorías

Vamos a ver cómo manejar categorías mal escritas usando diferentes técnicas:

In [ ]:
# Crear datos de ejemplo con categorías mal escritas
categorias_ejemplo = pd.DataFrame({
    'categoria': [
        'Electronica',
        'electronica',
        'Electrónica',
        'ELECTRONICA',
        'Electrnica',
        'Ropa',
        'ropa',
        'ROPA',
        'Roppa',
        'Vestimenta',
        'Deportes',
        'deportes',
        'DEPORTES',
        'Deports',
        'Sports'
    ]
})

print("Categorías originales:")
print(categorias_ejemplo['categoria'].value_counts())

In [ ]:
def estandarizar_categoria(categoria, categorias_correctas, umbral=80):
    """Función para estandarizar categorías usando fuzzy matching"""
    mejor_match = None
    mejor_score = 0
    
    for cat_correcta in categorias_correctas:
        # Calcular similitud usando ratio de Levenshtein
        score = fuzz.ratio(categoria.lower(), cat_correcta.lower())
        
        if score > mejor_score and score >= umbral:
            mejor_score = score
            mejor_match = cat_correcta
    
    return mejor_match if mejor_match else 'Otros'

# Definir categorías correctas
categorias_correctas = ['Electrónica', 'Ropa', 'Deportes']

# Aplicar la estandarización
categorias_ejemplo['categoria_limpia'] = categorias_ejemplo['categoria'].apply(
    lambda x: estandarizar_categoria(x, categorias_correctas)
)

print("\nCategorías después de la limpieza:")
print(categorias_ejemplo)

<a id='3'></a>
## 3. Detección y Manejo de Outliers

Vamos a ver diferentes técnicas para detectar y manejar valores atípicos:

In [ ]:
# Crear datos de ejemplo con outliers
np.random.seed(42)
n = 1000

# Generar datos normales
datos_normales = np.random.normal(100, 15, n)

# Añadir algunos outliers
outliers = np.random.uniform(200, 300, 20)
datos = np.concatenate([datos_normales, outliers])

df_outliers = pd.DataFrame({
    'valor': datos
})

# Visualizar distribución
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_outliers['valor'])
plt.title('Distribución de valores con outliers')
plt.show()

In [ ]:
def detectar_outliers_iqr(df, columna):
    """Detectar outliers usando el método IQR"""
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = df[
        (df[columna] < limite_inferior) | 
        (df[columna] > limite_superior)
    ]
    
    return outliers, limite_inferior, limite_superior

def detectar_outliers_zscore(df, columna, umbral=3):
    """Detectar outliers usando Z-score"""
    z_scores = np.abs(stats.zscore(df[columna]))
    return df[z_scores > umbral]

# Detectar outliers usando IQR
outliers_iqr, limite_inf, limite_sup = detectar_outliers_iqr(df_outliers, 'valor')

print(f"Número de outliers detectados (IQR): {len(outliers_iqr)}")
print(f"Límite inferior: {limite_inf:.2f}")
print(f"Límite superior: {limite_sup:.2f}")

# Limpiar outliers (ejemplo con winsorización)
df_outliers['valor_limpio'] = df_outliers['valor'].clip(limite_inf, limite_sup)

# Visualizar resultado
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

sns.boxplot(x=df_outliers['valor'], ax=ax1)
ax1.set_title('Datos originales con outliers')

sns.boxplot(x=df_outliers['valor_limpio'], ax=ax2)
ax2.set_title('Datos después de la limpieza')

plt.tight_layout()
plt.show()

<a id='4'></a>
## 4. Validación de Datos

Vamos a implementar algunas funciones de validación para asegurar la calidad de los datos:

In [ ]:
def validar_email(email):
    """Validar formato de email"""
    patron = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    return bool(re.match(patron, str(email)))

def validar_telefono(telefono):
    """Validar formato de teléfono (ejemplo para España)"""
    patron = r'^(\+34|0034|34)?[6789]\d{8}$'
    return bool(re.match(patron, str(telefono)))

def validar_dni(dni):
    """Validar formato de DNI español"""
    patron = r'^\d{8}[A-Z]$'
    return bool(re.match(patron, str(dni)))

# Crear datos de ejemplo para validación
datos_validacion = pd.DataFrame({
    'email': [
        'usuario@dominio.com',
        'invalido@.com',
        'usuario@dominio',
        'usuario.nombre@dominio.com'
    ],
    'telefono': [
        '666777888',
        '+34666777888',
        '12345',
        '666-777-888'
    ],
    'dni': [
        '12345678Z',
        '1234567',
        '12345678z',
        'A12345678'
    ]
})

# Aplicar validaciones
datos_validacion['email_valido'] = datos_validacion['email'].apply(validar_email)
datos_validacion['telefono_valido'] = datos_validacion['telefono'].apply(validar_telefono)
datos_validacion['dni_valido'] = datos_validacion['dni'].apply(validar_dni)

print("Resultados de la validación:")
print(datos_validacion)

## Conclusiones de la Parte 2

En esta segunda parte hemos cubierto técnicas más avanzadas de limpieza de datos:
1. Limpieza de fechas usando expresiones regulares
2. Estandarización de categorías usando fuzzy matching
3. Detección y manejo de outliers
4. Validación de datos con regex

En la Parte 3 continuaremos con:
- Manejo de datos faltantes con técnicas avanzadas
- Normalización y escalado de datos
- Técnicas de imputación
- Casos prácticos completos